РГР 2
Чу Ван Доан - Нгуен Тхе Вьет

In [1]:
!python --version

Python 3.10.12


#Часть 1

Импортируем нужные библиотеки

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.stats import chisquare, kstest, uniform
from google.colab import files
uploaded = files.upload()

Saving mobile_phones.csv to mobile_phones.csv


Считываем датасет

In [ ]:
file_path = 'mobile_phones.csv'
df = pd.read_csv(file_path)
df

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


# Часть 1. Задание 1

Гипотеза $H_0$ - емкость аккумулятора согласована с равномерным законом

Гипотеза $H_1$ - обратное

Разобьём имеющиеся значения емкости аккумулятора на 10 интервалов

In [ ]:
count = [0]*10

for i in range(10):
  lowerEdge = 501 + i*(1998-501)/10
  higherEdge = lowerEdge + (1998-501)/10
  count[i] = len(df[(df.battery_power >= lowerEdge) & (df.battery_power <= higherEdge)])
  print(i, ":", lowerEdge, "->", higherEdge, ":", count[i])

0 : 501.0 -> 650.7 : 223
1 : 650.7 -> 800.4000000000001 : 199
2 : 800.4 -> 950.0999999999999 : 210
3 : 950.1 -> 1099.8 : 199
4 : 1099.8 -> 1249.5 : 190
5 : 1249.5 -> 1399.2 : 185
6 : 1399.2 -> 1548.9 : 193
7 : 1548.9 -> 1698.6000000000001 : 201
8 : 1698.6 -> 1848.3 : 197
9 : 1848.3 -> 1998.0 : 203


Получим 10 интервалов. Будем считать, что вероятность попадания в интервал равна $\frac{1}{10}$, тогда в формуле ниже $n*p_i$ будет равняться 200.

Вычислим $\chi^2_{наб} = ∑ \dfrac{(v_i-n*p_i)^2}{n*p_i} $

In [ ]:
xi1 = 0
for i in range(10):
  xi1 += (((count[i] - 200)**2)/200)
print("Хи-квадрат наблюдаемый равен:", xi1)

Хи-квадрат наблюдаемый равен: 5.119999999999999


По таблице найдём $\chi^2_{cr}$. Имеем 10 промежутков, поэтому необходимо выбрать $\chi^2_{9}$. Уровень значимости будем считать равным 0.05.

Получим $\chi^2_{cr}=16.9$

$\chi^2_{наб} < \chi^2_{cr} ⇒ H_0$ принимается

Решим задачу при помощи другого критерия. Воспользуемся критерием Колмогорова, гипотезы оставим те же.

In [ ]:
from scipy.stats import kstest

min_val = 501
max_val = 1998
battery_capacity_normalized = (df['battery_power'] - min_val) / (max_val - min_val)

ks_stat, p_value_ks = kstest(battery_capacity_normalized, 'uniform')

print("Статистика Колмогорова-Смирнова:", ks_stat)

Статистика Колмогорова-Смирнова: 0.02077087508350034


По таблице найдём $\chi^2_{cr}$. Уровень значимости будем считать равным 0.05.

Получим $\chi^2_{cr}=1.358$

$\chi^2_{наб} < \chi^2_{cr} ⇒ H_0$ принимается.

# Часть 1. Задание 2

Гипотеза $H_0$ - распределения выборок одинаковые

Гипотеза $H_1$ - обратное

Разобъем датасет на два по принципу наличия/отсутствия wifi

In [ ]:
haveWifi = df[df.wifi == 1]
notWifi = df[df.wifi == 0]
print(len(haveWifi), len(notWifi))

1014 986


Как и в прошлой задаче, разобьём имеющиеся значения емкости аккумулятора на 10 интервалов

In [ ]:
countWifi = [0]*10
countNWifi = [0]*10

for i in range(10):
  lowerEdge = 501 + i*(1998-501)/10
  higherEdge = lowerEdge + (1998-501)/10
  countWifi[i] = len(haveWifi[(haveWifi.battery_power >= lowerEdge) & (haveWifi.battery_power <= higherEdge)])
  countNWifi[i] = len(notWifi[(notWifi.battery_power >= lowerEdge) & (notWifi.battery_power <= higherEdge)])
  print(i, ":", lowerEdge, "->", higherEdge, ":", countWifi[i], countNWifi[i])

0 : 501.0 -> 650.7 : 107 116
1 : 650.7 -> 800.4000000000001 : 109 90
2 : 800.4 -> 950.0999999999999 : 111 99
3 : 950.1 -> 1099.8 : 95 104
4 : 1099.8 -> 1249.5 : 94 96
5 : 1249.5 -> 1399.2 : 99 86
6 : 1399.2 -> 1548.9 : 105 88
7 : 1548.9 -> 1698.6000000000001 : 95 106
8 : 1698.6 -> 1848.3 : 101 96
9 : 1848.3 -> 1998.0 : 98 105


Найдем вероятности

In [ ]:
probHere = [0]*10

for i in range(10):
  probHere[i] = (countWifi[i] + countNWifi[i])/2000
  print(probHere[i])

0.1115
0.0995
0.105
0.0995
0.095
0.0925
0.0965
0.1005
0.0985
0.1015


Вычислим хи-квадрат

In [ ]:
xi2 = 0

for i in range(10):
  xi2 += (countWifi[i]-len(haveWifi)*probHere[i])**2/(len(haveWifi)*probHere[i]) + (countNWifi[i]-len(notWifi)*probHere[i])**2/(len(notWifi)*probHere[i])

print("Хи-квадрат наблюдаемый равен:", xi2)

Хи-квадрат наблюдаемый равен: 6.281528077320096


По таблице найдём $\chi^2_{cr}$. Имеем 2 выборки и 10 интервалов емкости батареи, поэтому необходимо выбрать $\chi^2_{(1*9)}$. Уровень значимости будем считать равным 0.05.

Получим $\chi^2_{cr}=16.9$

$\chi^2_{наб} < \chi^2_{cr} ⇒ H_0$ принимается

Решим задачу при помощи другого критерия. Воспользуемся двувыборочным критерием Колмогорова-Смирнова, гипотезы оставим те же.

In [ ]:
from scipy.stats import ks_2samp

battery_capacity = df['battery_power']
wifi_support = df['wifi']

battery_wifi = battery_capacity[wifi_support == 1]
battery_no_wifi = battery_capacity[wifi_support == 0]

ks_stat, p_value_ks = ks_2samp(battery_wifi, battery_no_wifi)

print(f"Статистика Колмогорова-Смирнова: {ks_stat}")

Статистика Колмогорова-Смирнова: 0.028327552200231244


По таблице найдём $\chi^2_{cr}$. Уровень значимости будем считать равным 0.05.

Получим $\chi^2_{cr}=1.358$

$\chi^2_{наб} < \chi^2_{cr} ⇒ H_0$ принимается.

# Часть 1. Задание 3

Гипотеза $H_0$ - ценовая категория зависит от количества ядер

Гипотеза $H_1$ - обратное

Составим таблицу для 8 вариантов количества ядер процессора и 4 ценовых категорий

In [ ]:
table = [[0]*10]*4

for i in range(4):
  print("PR", i, end=': ')
  for j in range(1, 9):
    table[i][j] = len(df[(df.price_range == i) & (df.n_cores == j)])
    print(table[i][j], end='  ')
  print("sum:", sum(table[i]))
print("     ", end='')
for i in range(1, 9):
  print(len(df[(df.n_cores == i)]), end=' ')

PR 0: 49  69  62  67  59  61  66  67  sum: 500
PR 1: 76  59  69  76  51  54  55  60  sum: 500
PR 2: 52  57  56  73  66  57  69  70  sum: 500
PR 3: 65  62  59  58  70  58  69  59  sum: 500
     242 247 246 274 246 230 259 256 

Вычислим $\chi^2_{наб} = n \sum ∑ \dfrac{(v_{ij
}-(v_{i+}*v_{+j})/n)^2}{v_{i+}*v_{+j}} $

In [ ]:
xi3 = 0
for i in range(4):
  for j in range(1, 9):
    expected = sum(table[i])*len(df[df.n_cores == j])/2000
    xi3 += ((table[i][j] - expected)**2)/expected
print("Хи-квадрат наблюдаемый равен:", xi3)

Хи-квадрат наблюдаемый равен: 15.582260239670633


По таблице найдём $\chi^2_{cr}$. Имеем 4 ценовых категории и 8 вариантов количества ядер, поэтому необходимо выбрать $\chi^2_{(3*7)}$. Уровень значимости будем считать равным 0.05.

Получим $\chi^2_{cr}=32.7$

$\chi^2_{наб} < \chi^2_{cr} ⇒ H_0$ принимается

# Часть 2. Модель линейной регрессии

Считываем датасет

In [ ]:
uploaded = files.upload()
sf = pd.read_csv("fifa_players_stats.csv")
ff = sf[['Overall', 'Potential', 'Age', 'Value(in Euro)']]
ff

Saving fifa_players_stats.csv to fifa_players_stats.csv


,Overall,Potential,Age,Value(in Euro)
0,91,91,35,54000000
1,91,91,34,64000000
2,91,91,33,84000000
3,91,91,31,107500000
4,91,95,23,190500000
...,...,...,...,...
18534,47,56,21,110000
18535,47,57,17,90000
18536,47,67,18,130000
18537,47,61,17,100000


Импортируем нужные библиотеки

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy import stats

Разобъем данные на тренировочный и тестовый набор в соотношении 80/20.

После этого обучим модель линейной регрессии на тренировочном наборе, выведем коэффициенты.

Вычислим коэффициент детерминации для тестового набора.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ff.drop(['Value(in Euro)'], axis=1), ff['Value(in Euro)'], test_size=0.2)
LR = LinearRegression().fit(X_train, y_train)

print("Коэффициенты переменных:", LR.coef_)

cfd = LR.score(X_test, y_test)
print("Коэффициент детерминации:", cfd)

Коэффициенты переменных: [ 697417.84608056   85523.4188844  -366671.211054  ]
Коэффициент детерминации: 0.353907171037141


# Часть 2. Гипотезы

Сформулируем гипотезы для задач:
1. Чем меньше возраст, тем больше цена.

  Гипотеза $H_0$ - Коэффициент при возрасте $𝛽_{𝐴𝑔𝑒} ≥ 0$.
  
  Гипотеза $H_1$ - Коэффициент при возрасте $𝛽_{𝐴𝑔𝑒} < 0$.

In [ ]:
n = X_train.shape[0]  # число наблюдений
k = X_train.shape[1]  # число независимых переменных

# Коэффициент и стандартная ошибка для Age
age_coef = LR.coef_[2]

# Оценка стандартной ошибки коэффициентов
X_with_intercept = np.column_stack([np.ones(X_train.shape[0]), X_train])
se_model = np.sqrt(np.sum((y_train - LR.predict(X_train))**2) / (n - k - 1))
cov_matrix = np.linalg.inv(X_with_intercept.T @ X_with_intercept)
se_age = se_model * np.sqrt(cov_matrix[3, 3])

# t-статистика и p-значение
t_stat_age = age_coef / se_age
p_value_age = stats.t.sf(np.abs(t_stat_age), df=n-k-1)  # односторонний тест

alpha = 0.05
# Критическое значение
t_crit = stats.t.ppf(alpha, df=n-k-1)

print(f'Коэффициент при возрасте: {age_coef}')
print(f'Стандартная ошибка: {se_age}')
print(f't-статистика: {t_stat_age}')
print(f'Критическое значение: {t_crit}')
print(f'p-value: {p_value_age}')

if t_stat_age < t_crit and p_value_age < alpha:
    print("Отклоняем H_0 в пользу H_1: Цена зависит от возраста.")
else:
    print("Не можем отклонить H_0: Нет доказательств зависимости цены от возраста.")


Коэффициент при возрасте: -366671.211053996
Стандартная ошибка: 20805.251749238298
t-статистика: -17.62397376746091
Критическое значение: -1.64495640330117
p-value: 4.0528316531501015e-69
Отклоняем H_0 в пользу H_1: Цена зависит от возраста.


2. Цена зависит от рейтинга

  Гипотеза $H_0$ - Коэффициент при рейтинге $𝛽_{Overall}=0$

  Гипотеза $H_1$ - Коэффициент при рейтинге $𝛽_{Overall}\not=0$

In [ ]:
# Коэффициент и стандартная ошибка для Rating
rating_coef = LR.coef_[0]
se_rating = se_model * np.sqrt(cov_matrix[1, 1])

# t-статистика и p-значение
t_stat_rating = rating_coef / se_rating
p_value_rating = 2 * stats.t.sf(np.abs(t_stat_rating), df=n-k-1)

alpha = 0.05
# Критическое значение для двустороннего теста
t_crit = stats.t.ppf(1 - alpha/2, df=n-k-1)

print(f'Коэффициент при рейтинге: {rating_coef}')
print(f'Стандартная ошибка: {se_rating}')
print(f't-статистика: {t_stat_rating}')
print(f'Критическое значение: {t_crit}')
print(f'p-value: {p_value_rating}')

if abs(t_stat_rating) > t_crit and p_value_rating < alpha:
    print("Отклоняем H_0 в пользу H_1: Цена зависит от рейтинга.")
else:
    print("Не можем отклонить H_0: Нет доказательств зависимости цены от рейтинга.")


Коэффициент при рейтинге: 697417.8460805626
Стандартная ошибка: 18552.628695690473
t-статистика: 37.59132236838025
Критическое значение: 1.9601239940904691
p-value: 1.7743619379064536e-295
Отклоняем H_0 в пользу H_1: Цена зависит от рейтинга.


3. Цена одновременно зависит от потенциального рейтинга и возраста

  Гипотеза $H_0$ - Коэффициенты при потенциальном рейтинге и возрасте $𝛽_{Potential} = 0$ и $𝛽_{𝐴𝑔𝑒} = 0$
  
  Гипотеза $H_1$ - Коэффициенты при потенциальном рейтинге и возрасте не равны нулю.

In [ ]:
# Оставляем только потенциальный рейтинг и возраст
X_reduced = X_train[['Potential', 'Age']]

# Строим модель с ограничением
reduced_model = LinearRegression().fit(X_reduced, y_train)
y_reduced_pred = reduced_model.predict(X_reduced)

# Полная модель
y_full_pred = LR.predict(X_train)

# Число наблюдений и число предикторов
n = X_train.shape[0]
p_full = X_train.shape[1]  # полная модель
p_reduced = X_reduced.shape[1]  # ограниченная модель

# RSS для полной и ограниченной модели
rss_full = np.sum((y_train - y_full_pred) ** 2)
rss_reduced = np.sum((y_train - y_reduced_pred) ** 2)

# F-статистика
f_statistic = ((rss_reduced - rss_full) / (p_full - p_reduced)) / (rss_full / (n - p_full))
f_p_value = stats.f.sf(f_statistic, dfn=(p_full - p_reduced), dfd=(n - p_full))

alpha = 0.05
f_crit = stats.f.ppf(1 - alpha, dfn=(p_full - p_reduced), dfd=(n - p_full))

print(f'F-статистика: {f_statistic}')
print(f'Критическое значение: {f_crit}')
print(f'p-value: {f_p_value}')

if f_statistic > f_crit and f_p_value < alpha:
    print("Отклоняем H_0 в пользу H_1: Цена зависит от потенциального рейтинга и возраста.")
else:
    print("Не можем отклонить H_0: Нет доказательств зависимости цены от потенциального рейтинга и возраста.")


F-статистика: 1413.2028237714985
Критическое значение: 3.842086029904047
p-value: 1.6953516087626318e-295
Отклоняем H_0 в пользу H_1: Цена зависит от потенциального рейтинга и возраста.
